In [1]:
import os
import fitz
import re
import json
from PyPDF2 import PdfReader

In [2]:
def extract_text_from_pdf(pdf_path, skip_pages=1):
    reader = PdfReader(pdf_path)
    text = ""
    for i, page in enumerate(reader.pages):
        if i < skip_pages:
            continue  # Пропускаем первые страницы
        text += page.extract_text() + "\n"
    return text

def parse_document(text, doc_title="Федеральные правила использования воздушного пространства РФ"):
    result = {
        "документ": doc_title,
        "главы": []
    }

    # Разделить по главам вида: I. Общие положения
    главы_raw = re.split(r"\n([IVXLCDM]+\.)\s+([^\n]+)", text)
    for i in range(1, len(главы_raw), 3):
        номер_главы = главы_raw[i]
        заголовок_главы = главы_raw[i+1].strip()
        текст_главы = главы_raw[i+2]

        # Ищем пункты: 1. ... (в начале строки)
        пункты = []
        for match in re.finditer(r"\n?(\d+)\.\s+(.*?)(?=\n\d+\.\s+|\Z)", текст_главы, re.DOTALL):
            номер = match.group(1)
            текст = re.sub(r"\s+", " ", match.group(2)).strip()
            пункты.append({
                "номер": f"{номер}.",
                "текст": текст
            })

        result["главы"].append({
            "глава": номер_главы,
            "заголовок": заголовок_главы,
            "пункты": пункты
        })

    return result



In [3]:
pdf_path = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_pdf\\11_03_2010_N_138.pdf"
outp_path = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_json\\11_03_2010_N_138.json"
doc_title = "ПОСТАНОВЛЕНИЕ от 11 марта 2010 г. N 138 'ОБ УТВЕРЖДЕНИИ ФЕДЕРАЛЬНЫХ ПРАВИЛ ИСПОЛЬЗОВАНИЯ ВОЗДУШНОГО ПРОСТРАНСТВА РОССИЙСКОЙ ФЕДЕРАЦИИ'"
text = extract_text_from_pdf(pdf_path, skip_pages=1)
parsed_json = parse_document(text, doc_title)

with open(outp_path, "w", encoding="utf-8") as f:
    json.dump(parsed_json, f, ensure_ascii=False, indent=2)